## LLM zoomcamp Homework 1

### Q 1 

Run Elastic Search 8.17.6, and get the cluster information. What's the version.build_hash value?


```bash
docker stop $(docker ps -q)
docker rm $(docker ps -aq)

docker pull docker.elastic.co/elasticsearch/elasticsearch:8.17.6

docker run -p 9200:9200 -e "discovery.type=single-node" -e "ES_JAVA_OPTS=-Xms1g -Xmx1g" --memory="2g" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.17.6

In [17]:
from elasticsearch import Elasticsearch
import os 
from dotenv import load_dotenv
import requests

In [18]:
response = requests.get("http://localhost:9200")

In [19]:
data = response.json()


In [20]:
print(f"version.build_hash value is {data["version"]["build_hash"]} ")

version.build_hash value is dbcbbbd0bc4924cfeb28929dc05d82d662c527b7 


In [21]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q 2

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Which function do you use for adding your data to elastic?

In [28]:
import minsearch

/workspaces/llm-zoomcamp/week-1/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [ ]:
es_client = Elasticsearch('http://localhost:9200') 
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
index_name = "your_index_name"

mapping = {
    "mappings": {
        "properties": {
            "course": {"type": "keyword"},
            "question": {"type": "text"},
            "text": {"type": "text"},
            "section": {"type": "text"}
        }
    }
}

# Create the index with the mapping
es_client.indices.create(index=index_name, body=mapping)

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
es_client = Elasticsearch('http://localhost:9200') 

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [5]:
! curl http://localhost:9200

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (52) Empty reply from server
